## V2 
### Altair Stuff
* https://stackoverflow.com/questions/62109475/how-to-deselect-drop-down-box-of-altair

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp.sql import to_snakecase

In [ ]:
import altair as alt
import threshold_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils, rt_utils, styleguide, utils

In [ ]:
import intake
catalog = intake.open_catalog("./catalog_threshold.yml")

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Tests
* Keep name? 

In [ ]:
# Keep name?
def clean_trips():
    df = catalog.trips.read()

    subset = [
        "feed_key",
        "name",
        "route_id",
        "direction_id",
        "shape_id",
    ]

    df = df[subset]

    df = df.drop_duplicates().reset_index(drop=True)

    return df

In [ ]:
# trips = clean_trips()

In [ ]:
# trips.sample()

In [ ]:
def clean_routelines():
    df = catalog.route_lines.read()

    # Drop shape array key? Drop n_trips?
    df = df.drop(columns=["shape_array_key"])
    df = (df.drop_duplicates()).reset_index(drop=True)

    # Calculate length of geometry
    df = df.assign(actual_route_length=(df.geometry.length))

    return df

In [ ]:
# routelines = clean_routelines()

In [ ]:
# routelines.drop(columns = ['geometry']).sample(5)

In [ ]:
def clean_longest_shape():
    df = catalog.longest_shape.read()

    df = df.rename(columns={"route_length": "longest_route_length"})

    return df

In [ ]:
# longest_shape = clean_longest_shape()

In [ ]:
# Drop name?
# crosswalk = catalog.crosswalk.read()

In [ ]:
# crosswalk.sample()

In [ ]:
# Get outer merge counts including name
# pd.merge(trips, crosswalk, how="outer",
#         on = ["name","feed_key", "route_id", "direction_id"],
#         indicator=True)[["_merge"]].value_counts()

In [ ]:
# Get outer merge counts excluding name
# pd.merge(trips, crosswalk, how="outer",
#         on = ["feed_key", "route_id", "direction_id"],
#         indicator=True)[["_merge"]].value_counts()

In [ ]:
# m1 = trips.merge(crosswalk, how="inner", on=["feed_key", "route_id", "name", "direction_id"])

In [ ]:
# routelines.sample()

In [ ]:
# m2 = m1.merge(routelines, how="inner", on=["feed_key", "shape_id"])

In [ ]:
# m3.sample(3).drop(columns = ['geometry','geometry_arrowized'])

In [ ]:
# m3.gtfs_dataset_key.nunique(), m3.name.nunique()

### Function
* Best way is using `gtfs_dataset_key` or `name`?

In [ ]:
def merge_trips_routes_longest_shape():
    trips = clean_trips()
    crosswalk = catalog.crosswalk.read()
    routelines = clean_routelines()
    longest_shape = clean_longest_shape()

    m1 = (
        trips.merge(
            crosswalk, how="inner", on=["feed_key", "route_id", "name", "direction_id"]
        )
        .merge(routelines, how="inner", on=["feed_key", "shape_id"])
        .merge(
            longest_shape.drop(columns=["geometry"]),
            how="inner",
            on=[
                "feed_key",
                "gtfs_dataset_key",
                "direction_id",
                "route_id",
                "route_dir_identifier",
                "name",
            ],
        )
    )

    # Calculate out proportion of route length against longest.
    m1["route_length_percentage"] = (
        (m1["actual_route_length"] / m1["longest_route_length"]) * 100
    ).astype(int)

    # Count number of segments that appear in the longest shape.
    m1 = (
        m1.groupby(
            [
                "route_id",
                "name",
                "gtfs_dataset_key",
                "route_dir_identifier",
                "shape_id",
                "longest_shape_id",
                "route_length_percentage",
            ]
        )
        .agg({"segment_sequence": "count"})
        .rename(columns={"segment_sequence": "total_segments"})
        .reset_index()
    )

    return m1

In [ ]:
def calculate_longest_shape(operator: str):
    """
    Get total segments of the longest shape and
    shape_id's length compared to the longest
    shape_id's length for a route.
    """
    df = merge_trips_routes_longest_shape()

    df = df.loc[df.name == operator].reset_index(drop=True)

    agg = (
        df.groupby(["name", "route_id", "shape_id"])
        .agg({"route_length_percentage": "max"})
        .reset_index()
    )

    return agg

In [ ]:
def summary_stats_route_length():
    """
    Get mean, median, max, and min
    of route length for each operator
    """
    df = merge_trips_routes_longest_shape()

    df = (
        df.groupby(["name", "route_id", "shape_id"])
        .agg({"route_length_percentage": "max"})
        .reset_index()
    )

    df = (
        df.groupby("name")
        .agg({"route_length_percentage": ["mean", "median", "min", "max"]})
        .reset_index()
    )

    df.columns = df.columns.droplevel()

    df = df.rename(columns={"": "Name"})

    df = pd.melt(df, id_vars=["Name"], value_vars=["mean", "median", "min", "max"])

    df.variable = df.variable.str.title()
    
    return df

In [ ]:
route_length = summary_stats_route_length()

In [ ]:
route_length.sample(5)

In [ ]:
# route_length = pd.melt(route_length, id_vars=['Name'], value_vars=['mean', 'median','min','max'])

### Operator

* just 1 boxplot or dot plot for the operator for scheduled shapes % route length
* i've also pared back the summary stats to accompany each chart (just the crucial stuff)
* only allow altair selection on operator, not time or segment, i'm not going to remember what it is moving from selection to selection, so i'd rather see it at once. 
    * For usable trips remove time/segment. 
    * Thought the workflow would be running per operator as opposed to running all operators and selecting from the dropdown menu?
    * If the dropdown menu should be operator -> have to combine all the dataframes together.
* if you find it too crowded, i suggest paring down what's * included in SEGMENT_CUTOFFS ....likely we're never going to go up to 0.9, so you can probably scale it back
* continue with the alt.vconcat, but i want the single boxplot or dotplot to be aligned the same way as the bar
the table can be printed below (outside of the charts) if that's easier
    * Wants charts to run in the same direction.
    * Rotate dotplot instead of increasing up and down. Increasing should be left to right. 
    

* Box plot looks super bare.

#### Shape ID vs. Longest Shape ID Lengths 
* Play with jitter, opacity, size. 
    * Jitter looks exactly the same.

In [ ]:
def length_comparison_dot(df):

    df = threshold_utils.clean_up_columns(df)

    # Dropdown menu 1
    dropdown1 = alt.binding_select(
        options=df["Name"].sort_values().unique().tolist(), name="Name"
    )
    selection1 = alt.selection_single(fields=["Name"], bind=dropdown1)

    chart = (
        alt.Chart(df, width=5)
        .mark_circle(size=200)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
           y=alt.Y("Value:Q"),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=None,
            ),
            tooltip=[
                "Name",
                "Variable",
                "Value",
            ],
            column=alt.Column(
                "Name:N",
                header=alt.Header(
                    labelAngle=90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title=f"Routes by Length for Operator")
        .add_selection(selection1)
        .transform_filter(selection1)
    )
    return chart 

In [ ]:
# length_comparison_dot(route_length)

In [ ]:
# Dropdown menu 1
dropdown1 = alt.binding_select(
        options=route_length["Name"].sort_values().unique().tolist(), name="Name"
    )
selection1 = alt.selection_single(fields=["Name"], bind=dropdown1)

In [ ]:
# https://altair-viz.github.io/gallery/wilkinson-dot-plot.html
threshold_utils.chart_size(alt.Chart(route_length).mark_circle(opacity=1, size = 100).transform_window(
    id='rank()',
    groupby=['Variable']
).encode(
    alt.Y('Name:N'),
    alt.X('Value:O', sort='ascending', axis=alt.Axis(ticks=False, grid=False)), 
    color=alt.Color("Variable:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)),
    tooltip = ['Name', 'Variable', 'Value']
).add_selection(selection1)
.transform_filter(selection1), 1000,1000)

In [ ]:
boxplot = merge_trips_routes_longest_shape()

In [ ]:
boxplot = (
    boxplot.groupby(["name", "route_id", "shape_id"])
    .agg({"route_length_percentage": "max"})
    .reset_index()
)

In [ ]:
threshold_utils.chart_size(
    (
        alt.Chart(boxplot)
        .mark_boxplot(extent="min-max")
        .encode(
            x="route_length_percentage:Q",
            y="name:N",
            color=alt.Color(
                "name",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=None,
            ),
        )
    ).properties(title=f"Test"),
    1000,
    1000,
)

In [ ]:
threshold_utils.chart_size(
    (
        alt.Chart(boxplot.loc[boxplot.name == 'Redding Schedule'])
        .mark_boxplot(extent="min-max")
        .encode(
            x="route_length_percentage:Q",
            y="name:N",
            color=alt.Color(
                "name",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=None,
            ),
        )
    ).properties(title=f"Test"),
    500,
    300,
)

#### Trip Time and Segment

In [ ]:
def merge_trip_diagnostics_with_total_segments():
    trip_diagnostics = catalog.trip_stats.read()
    segments = catalog.longest_shape.read()

    total_segments_by_shape = (
        segments.groupby(["gtfs_dataset_key", "name", "route_dir_identifier"])
        .segment_sequence.nunique()
        .reset_index()
        .rename(columns={"segment_sequence": "total_segments"})
    )

    df = pd.merge(
        trip_diagnostics,
        total_segments_by_shape,
        on=["gtfs_dataset_key", "route_dir_identifier"],
        how="inner",
        validate="m:1",
    )

    # Should  this be divided by 60 to be in minutes for the latter chunk of code?
    df = df.assign(
        pct_vp_segments=df.num_segments_with_vp.divide(df.total_segments),
        trip_time=((df.trip_end - df.trip_start) / np.timedelta64(1, "s") / 60).astype(
            int
        ),
        total_trips=df.groupby(["gtfs_dataset_key", "name"]).trip_id.transform(
            "nunique"
        ),
    )

    return df

In [ ]:
all_ops = merge_trip_diagnostics_with_total_segments()

In [ ]:
def summary_valid_trips_by_cutoff(df, time_cutoffs: list, segment_cutoffs: list):
    final = pd.DataFrame()

    for t in time_cutoffs:
        for s in segment_cutoffs:
            valid = (
                df[(df.trip_time >= t) & (df.pct_vp_segments >= s)]
                .groupby(["gtfs_dataset_key", "name", "total_trips"])
                .trip_id.nunique()
                .reset_index()
                .rename(columns={"trip_id": "n_trips"})
            )

            valid = valid.assign(
                trip_cutoff=t, segment_cutoff=s, cutoff=f"{t}+ min & {s*100}%+ segments"
            )

            final = pd.concat([final, valid], axis=0)

    final = final.assign(pct_usable_trips=final.n_trips.divide(final.total_trips))

    return final

In [ ]:
TIME_CUTOFFS = [5, 10, 15]
SEGMENT_CUTOFFS = [
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.8
]

In [ ]:
valid_stats = summary_valid_trips_by_cutoff(all_ops, TIME_CUTOFFS, SEGMENT_CUTOFFS)

In [ ]:
def operator_trip_chart(df):
    df = threshold_utils.clean_up_columns(df)

    # Dropdown menu 1
    dropdown1 = alt.binding_select(
        options=df["Name"].sort_values().unique().tolist(), name="Name"
    )
    selection1 = alt.selection_single(fields=["Name"], bind=dropdown1)

    chart = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(
                "Pct Usable Trips:Q",
                sort=alt.SortField("Pct Usable Trips", order="descending"),
            ),
            y=alt.Y(
                "Cutoff:N", sort=alt.SortField("Pct Usable Trips", order="descending")
            ),
            color=alt.Color(
                "Cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
            ),
            tooltip=[
                "Name",
                "Cutoff",
                "Pct Usable Trips",
                "N Trips",
            ],
        )
        .properties(title=f"Percentage of Usable Trips")
        .add_selection(selection1)
        .transform_filter(selection1)
    )

    chart = threshold_utils.chart_size(chart, 500, 300)
    return chart

In [ ]:
operator_trip_chart(valid_stats)

### Statewide
#### Statewide Routes by Cutoff

In [ ]:
trips_routes_shape = merge_trips_routes_longest_shape()

In [ ]:
trip_stats = catalog.trip_stats.read()

In [ ]:
# trip_stats.columns, trips_routes_shape.columns

In [ ]:
def statewide_summary():
    trips_routes_shape = merge_trips_routes_longest_shape()
    trip_stats = catalog.trip_stats.read()

    m1 = trip_stats.merge(
        trips_routes_shape.drop(columns=["route_length_percentage"]),
        how="inner",
        on=["gtfs_dataset_key", "route_dir_identifier"],
    )

    m1 = m1.assign(
        pct_vp_segments=m1.num_segments_with_vp.divide(m1.total_segments),
        trip_time=((m1.trip_end - m1.trip_start) / np.timedelta64(1, "s")) / 60,
        total_routes=m1.groupby("route_id").trip_id.transform("nunique"),
    )

    return m1

In [ ]:
statewide = statewide_summary()

In [ ]:
statewide.head(3)

In [ ]:
def route_thresholds_time_only(df, time_cutoffs: list):

    total_unique_routes = df.route_id.nunique()

    # Find stats for all operators
    all_operators = pd.DataFrame()
    for t in time_cutoffs:
        valid = (
            df[(df.trip_time >= t)][["route_id"]]
            .nunique()
            .reset_index()
            .rename(columns={0: "Total Routes in Category"})
        )

        valid = valid.assign(route_cutoff=t, cutoff=f"{t}+ min")

        all_operators = pd.concat([all_operators, valid], axis=0)

    all_operators = all_operators.assign(
        total_routes=total_unique_routes,
        pct_usable_routes=all_operators["Total Routes in Category"].divide(
            total_unique_routes
        ),
        calitp_itp_id="all operators",
    )

    return all_operators

In [ ]:
statewide_thresholds_time_only = route_thresholds_time_only(statewide, TIME_CUTOFFS)

In [ ]:
statewide_thresholds_time_only.sample()

In [ ]:
threshold_utils.chart_size((
    alt.Chart(statewide_thresholds_time_only)
    .mark_bar()
    .encode(
        x=alt.X(
            "cutoff:N",
            sort=alt.SortField("pct_usable_routes", order="descending"),
        ),
        y=alt.Y(
            "pct_usable_routes:Q",
            sort=alt.SortField("pct_usable_routes", order="descending"),
        ),
        color=alt.Color(
            "cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
        ),
        tooltip=[
            "total_routes",
            "cutoff",
            "Total Routes in Category",
            "pct_usable_routes",
        ],
    )
    .properties(title=f"Percentage of Usable Routes for the State")
), 500, 300)

In [ ]:
def route_thresholds(df, time_cutoffs: list, segment_cutoffs: list):

    total_unique_routes = df.route_id.nunique()

    # Find stats for all operators
    all_operators = pd.DataFrame()

    for t in time_cutoffs:
        for s in segment_cutoffs:
            valid = (
                df[(df.trip_time >= t) & (df.pct_vp_segments >= s)][["route_id"]]
                .nunique()
                .reset_index()
                .rename(columns={0: "Total Routes in Category"})
            )

            valid = valid.assign(
                time_cutoff=t,
                segment_cutoff=s,
                route_cutoff=f"{t}+ min & {s*100}%+ segments",
                total_routes=total_unique_routes,
                pct_usable_routes=valid["Total Routes in Category"].divide(
                    total_unique_routes
                ),
            )

            all_operators = pd.concat([all_operators, valid], axis=0)

    return all_operators

In [ ]:
statewide_thresholds = route_thresholds(statewide, TIME_CUTOFFS, SEGMENT_CUTOFFS)

In [ ]:
(
    alt.Chart(statewide_thresholds)
    .mark_bar()
    .encode(
        x=alt.X(
            "pct_usable_routes:Q",
            sort=alt.SortField("pct_usable_routes", order="descending"),
        ),
        y=alt.Y(
            "route_cutoff:N",
            sort=alt.SortField("pct_usable_routes", order="descending"),
        ),
        color=alt.Color(
            "route_cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
        ),
        tooltip=[
            "total_routes",
            "route_cutoff",
            "Total Routes in Category",
            "pct_usable_routes",
        ],
    )
    .properties(title=f"Percentage of Usable Routes for the State")
)

#### Route Thresholds by Operators

In [ ]:
# Find stats for all operators
operator_df = pd.DataFrame()

In [ ]:
# Subset statewide to only 2 operators
subset = statewide[statewide["name"].isin(['Big Blue Bus Schedule','Bay Area 511 AC Transit Schedule',])].reset_index(drop = True)

In [ ]:
for o in subset.name.unique().tolist():
    for t in TIME_CUTOFFS:
        for s in SEGMENT_CUTOFFS:
            valid = subset.loc[subset.name == o].reset_index(drop = True)
            total_unique_routes = valid.route_id.nunique() 
            operator = valid.head(1).iloc[0]["name"]
            valid = (
                    valid[(valid.trip_time >= t) & (valid.pct_vp_segments >= s)][["route_id"]]
                    .nunique()
                    .reset_index()
                    .rename(columns={0: "Total Routes in Category"})
                )

            valid = valid.assign(
                time_cutoff=t,
                segment_cutoff=s,
                name = operator,
                route_cutoff=f"{t}+ min & {s*100}%+ segments",
                total_routes=total_unique_routes,
                pct_usable_routes=valid["Total Routes in Category"].divide(
                    total_unique_routes
                ),
            )

            operator_df = pd.concat([operator_df, valid], axis=0)

In [ ]:
empty_df.name.sort_values().unique().tolist()

In [ ]:
# Dropdown menu 1
dropdown1 = alt.binding_select(
        options=empty_df["name"].sort_values().unique().tolist(), name="name"
    )
selection1 = alt.selection_single(fields=["name"], bind=dropdown1)

(
        alt.Chart(empty_df)
        .mark_bar()
        .encode(
            x=alt.X(
                "pct_usable_routes:Q",
                sort=alt.SortField("pct_usable_routes", order="descending"),
            ),
            y=alt.Y(
                "route_cutoff:N", sort=alt.SortField("pct_usable_routes", order="descending")
            ),
            color=alt.Color(
                "route_cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
            ),
            tooltip=[

            ],
        )
        .properties(title=f"Percentage of Usable Routes")
    )#.add_selection(selection1).transform_filter(selection1)

In [ ]:
# Find stats for all operators
operator_df2 = pd.DataFrame()

In [ ]:
for o in statewide.name.unique().tolist():
    for t in TIME_CUTOFFS:
            valid = statewide.loc[statewide.name == o].reset_index(drop = True)
            total_unique_routes = valid.route_id.nunique() 
            operator = valid.iloc[0]["name"]
            valid = (
                    valid[(valid.trip_time >= t)][["route_id"]]
                    .nunique()
                    .reset_index()
                    .rename(columns={0: "Total Routes in Category"})
                )

            valid = valid.assign(
                time_cutoff=t,
                segment_cutoff=s,
                name = operator,
                route_cutoff=f"{t}+ min",
                total_routes=total_unique_routes,
                pct_usable_routes=valid["Total Routes in Category"].divide(
                    total_unique_routes
                ),
            )

            operator_df2 = pd.concat([operator_df2, valid], axis=0)

In [ ]:
def operator_trip_chart(df):
    df = threshold_utils.clean_up_columns(df)

    # Dropdown menu 1
    dropdown1 = alt.binding_select(
        options=df["Name"].sort_values().unique().tolist(), name="Name"
    )
    selection1 = alt.selection_single(fields=["Name"], bind=dropdown1)

    chart = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(
                "Pct Usable Routes:Q",
                sort=alt.SortField("Pct Usable Routes", order="descending"),
            ),
            y=alt.Y(
                "Route Cutoff:N", sort=alt.SortField("Pct Usable Routes", order="descending")
            ),
            color=alt.Color(
                "Route Cutoff:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS)
            ),
            tooltip=[
                "Name",
                "Route Cutoff",
                "Pct Usable Routes",
                "Total Routes",
                "Total Routes In Category"
            ],
        )
        .properties(title=f"Percentage of Usable Trips")
        .add_selection(selection1)
        .transform_filter(selection1)
    )

    chart = threshold_utils.chart_size(chart, 500, 300)
    return chart

In [ ]:
operator_trip_chart(operator_df2)